# Cross validation of the ALS method using Spark
We refer to this tutorial: http://spark.apache.org/docs/latest/ml-tuning.html#example-model-selection-via-cross-validation

The tutorial was not clear: we used SKLearn to split the dataset

# Import

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import itertools
from pyspark.mllib.recommendation import ALS
import math

%matplotlib inline
%load_ext autoreload
%autoreload 2

pd.options.display.max_columns = 100


In [17]:
from pyspark.sql.functions import col

In [3]:
from helpers_cross_validation import KFoldIndexes,CrossValidation

In [4]:
import random


# Models

In [11]:
class ALSModel:
    
    def __init__(self):
        pass
    
    def fit(self,data,**arg):
        self.model = ALS.train(data, **arg)
    
    def predict(self,data):
        data_for_preditions=data.map(lambda x: (x[0], x[1]))
        self.predictions = self.model.predictAll(data_for_preditions).map(lambda r: ((r[0], r[1]), r[2]))
    
    def evaluate(self,data):
        rates_and_preds = data.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(self.predictions)
        error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
        return error

# Dataframe creation

In [6]:
train = pd.read_csv('../data/data_train.csv')


In [7]:
train['UserID'] = train['Id'].apply(lambda x: int(x.split('_')[0][1:]))
train['MovieID'] = train['Id'].apply(lambda x: int(x.split('_')[1][1:]))
train['Rating'] = train['Prediction']
train = train.drop(['Id', 'Prediction'], axis=1)

In [8]:
train.shape[0]

1176952

# Cross validation

In [9]:
cv=CrossValidation(train,4,True,sc)

In [12]:
cv.evaluate(ALSModel(),rank=2,lambda_=0.1, iterations=20)

[0.9983470606516812,
 0.9986656143117899,
 1.0006664314237084,
 1.0014538353492282]

# Others

In [ ]:
tests=get_tests_database(train,k_fold_indexes)

In [ ]:
tests_sql=get_sql_from_pd(tests)

In [ ]:
type(tests[0])

In [ ]:
sc.union(tests_sql)

In [ ]:
tr=tests_sql[0].rdd.union(tests_sql[1].rdd).union(tests_sql[2].rdd)

In [ ]:
ts=tests_sql[3].rdd

In [ ]:
model = ALS.train(tr, 2, 10, 0.1)

In [ ]:
validation_for_predict_RDD = ts.map(lambda x: (x[0], x[1]))

In [ ]:
predictions = model.predictAll(validation_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))

In [ ]:
rates_and_preds = ts.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)

In [ ]:
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())

In [ ]:
error

In [ ]:
tests_sql

In [ ]:
train_sql = sqlContext.createDataFrame(train)
train_rdd = train_sql.rdd
train_rdd.take(1)

In [ ]:
type(train_sql.map(lambda x: (x[0],x[1],float(x[2]))))